## **Computer vision project: Deggendorf Waste Sorting Assistant**

### **Overview**
The Deggendorf Waste Sorting Assistant is a Computer Vision-based tool designed to help residents and international students correctly identify waste bins. The project leverages image classification to determine the category of a given waste bin based on its visual characteristics. Users can take a picture of an unlabeled bin, and the model will classify it while providing information on the appropriate waste materials for disposal.

### **Project Goals**
- Develop an image classification model capable of identifying waste bins in Deggendorf.
- Provide users with clear guidance on proper waste disposal based on bin classification.
- Document all processes in a Jupyter Notebook, covering dataset creation, model training, evaluation, and deployment.


### 1. Mount Google Drive & Labeling Utility

This section mounts your Drive, sets up constants and logging, prepares a CSV to track labels, and provides an interactive widget in Colab that:

1. Lists all images in `MyDrive/cv_garbage` not yet labeled.  
2. Displays one image at a time.  
3. Lets you pick a label from a predefined list.  
4. Copies the image into `MyDrive/cv_garbage/labled` with a standardized filename.  
5. Appends metadata (`original_filename`, `new_filename`, `label`, `timestamp`) to `labels.csv`.


---

In [ ]:
# Run this cell first to install any missing dependencies
%pip install --upgrade pandas==2.2.2 pillow ipywidgets

#### 1.1 Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#### 1.2 Imports, Constants & Logging

In [ ]:
import os
import shutil
import logging
import pandas as pd
from datetime import datetime
from IPython.display import display, clear_output
from PIL import Image as PILImage
import ipywidgets as widgets

# ——— Logging ——————————————————————————————————————————————
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s: %(message)s"
)

# ——— Paths ——————————————————————————————————————————————
BASE_DIR    = '/content/drive/MyDrive/cv_garbage'
LABELED_DIR = os.path.join(BASE_DIR, 'labled')
CSV_PATH    = os.path.join(LABELED_DIR, 'labels.csv')
os.makedirs(LABELED_DIR, exist_ok=True)

# ——— Predefined Labels ——————————————————————————————————————
LABELS = [
    "Restmüll",     # residual waste
    "Biomüll",      # organic waste
    "Papier",       # paper
    "Gelber_Sack",  # packaging
    "Glas"          # glass
]

#### 1.3 Colab Image Labeler Class

In [ ]:
class ColabImageLabeler:
    """
    Interactive Colab image labeler:
    - Reads images from BASE_DIR.
    - Skips already labeled images (tracked in CSV).
    - Displays one image at a time.
    - On label selection, copies image to LABELED_DIR with new name.
    - Appends metadata to CSV.
    """
    def __init__(self, src_dir, dst_dir, csv_path, labels):
        self.src_dir  = src_dir
        self.dst_dir  = dst_dir
        self.csv_path = csv_path
        self.labels   = labels
        self.df       = self._load_or_init_csv()
        self.images   = self._get_unlabeled_images()
        self.index    = 0

        # Widgets
        self.out_img    = widgets.Output()
        self.dropdown   = widgets.Dropdown(options=self.labels, description='Label:')
        self.next_btn   = widgets.Button(description='Next ▶️')
        self.status_lbl = widgets.Label()

        self.next_btn.on_click(self._on_next)

    def _load_or_init_csv(self):
        """Load existing CSV or create new DataFrame."""
        if os.path.exists(self.csv_path):
            df = pd.read_csv(self.csv_path)
            logging.info(f"Loaded {len(df)} labeled entries from CSV.")
        else:
            df = pd.DataFrame(columns=[
                'original_filename',
                'new_filename',
                'label',
                'timestamp'
            ])
            logging.info("Initialized new labeling CSV.")
        return df

    def _get_unlabeled_images(self):
        """List images in src_dir not yet labeled."""
        exts = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
        all_imgs = sorted(
            [f for f in os.listdir(self.src_dir)
             if os.path.splitext(f.lower())[1] in exts]
        )
        labeled = set(self.df['original_filename'])
        unlabeled = [f for f in all_imgs if f not in labeled]
        logging.info(f"{len(unlabeled)} unlabeled images found.")
        return unlabeled

    def _show_image(self, filepath):
        """Display image in the output widget."""
        with self.out_img:
            clear_output(wait=True)
            display(PILImage.open(filepath).resize((480, 360)))

    def _on_next(self, _):
        """Handle Next button: copy, record, advance."""
        if self.index >= len(self.images):
            return

        fname = self.images[self.index]
        src   = os.path.join(self.src_dir, fname)
        label = self.dropdown.value.replace(' ', '_')
        ext   = os.path.splitext(fname)[1]
        timestamp = datetime.utcnow().isoformat()
        # Zero-padded index for stable ordering
        new_name  = f"{label}_{self.index+1:04d}{ext}"
        dst       = os.path.join(self.dst_dir, new_name)

        try:
            shutil.copy2(src, dst)
            logging.info(f"✅  Copied {fname} → {new_name}")
            # Append metadata
            self.df = self.df.append({
                'original_filename': fname,
                'new_filename'     : new_name,
                'label'            : label,
                'timestamp'        : timestamp
            }, ignore_index=True)
            self.df.to_csv(self.csv_path, index=False)
            self.status_lbl.value = f"Labeled: {new_name}"
        except Exception as e:
            logging.error(f"❌  Error processing {fname}: {e}")
            self.status_lbl.value = f"Error: {e}"

        self.index += 1
        if self.index < len(self.images):
            self._render_current()
        else:
            clear_output(wait=True)
            print("🎉  Done! All images labeled. Metadata saved to CSV.")

    def _render_current(self):
        """Render current image + status."""
        fname = self.images[self.index]
        path  = os.path.join(self.src_dir, fname)
        self._show_image(path)
        self.status_lbl.value = f"Image {self.index+1}/{len(self.images)}: {fname}"

    def start(self):
        """Launch the interactive UI."""
        if not self.images:
            print("No images to label. All done!")
            return
        self._render_current()
        ui = widgets.VBox([
            self.out_img,
            widgets.HBox([self.dropdown, self.next_btn]),
            self.status_lbl
        ])
        display(ui)

#### 1.4 Launch Labeler

In [ ]:
if __name__ == '__main__':
    labeler = ColabImageLabeler(
        src_dir=BASE_DIR,
        dst_dir=LABELED_DIR,
        csv_path=CSV_PATH,
        labels=LABELS
    )
    labeler.start()

### 2. Import Required Libraries for the Rest of the Project

In [ ]:
# 1. Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 3. Dataset Creation

In [ ]:
# TODO: code for splitting data, creating train/val folders, etc. …

### 4. Model Training

In [ ]:
# TODO: code for defining and training your CNN …

### 5. Evaluation & Deployment

In [ ]:
# TODO: code for evaluating accuracy, exporting a TensorFlow Lite model, etc. …